# Data Sharing with Encryption - ControlER side
### Imports & W/R byte functions


In [ ]:
from Crypto.Cipher import AES
from Crypto import Random
import pandas as pd
import os
from cryptography.hazmat.backends import default_backend
import socket

HOST = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)
# get current directory
path = os.getcwd()
parent = os.path.dirname(path)

DIR_DATA = parent+ '/data/'
        
def write_file(data,filename):
    with open(filename, 'wb') as f:
        f.write(data)
        f.close()

def read_file(filename):
    with open(filename, 'rb') as f:
        data_b = f.read()
        f.close()
    return data_b

### Data Description

In [ ]:
df_columns = pd.read_csv(DIR_DATA+'columns_description.csv')
df_columns.head()

In [ ]:
df_columns.info()

In [ ]:
df_infringment = pd.read_csv(DIR_DATA+'infringement_dataset.csv')
df_infringment.head()

In [ ]:
df_infringment.info()

### Public Key exchange

#### DH (Diffie–Hellman)

- It uses very large numbers and a lot of math, something that many of us still dread from those long and boring high school lessons
- On a mathematical level, the Diffie-Hellman key exchange relies on one-way functions as the basis for its security. These are calculations which are simple to do one way, but much more difficult to calculate in reverse.
- In the real world, the Diffie-Hellman key exchange is rarely used by itself. The main reason behind this is that it **provides no authentication**, which leaves users vulnerable to **man-in-the-middle attacks**
- You could generate the parameters on the server and send them to the client, but a better option is to use a set of predefined parameters, for example [group 14 defined in RFC 3526](https://www.ietf.org/rfc/rfc3526.txt)

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat,load_der_public_key

p = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF
g = 2

params_numbers = dh.DHParameterNumbers(p,g)
parameters = params_numbers.parameters(default_backend())

In [ ]:
def dh_key_exchange(params):
    key_A = params.generate_private_key() # red
    public_mix_A  = key_A.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)  #orange
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print('Waiting for connection...')
        conn, addr = s.accept()

        with conn:
            print(f"Connected by {addr}")
            conn.sendall(public_mix_A)
            public_mix_B = conn.recv(1024)
            print(f'Received public key in bytes from Controler {public_mix_B}')
        conn.close()

    public_mix_B_ser = load_der_public_key(public_mix_B,backend = default_backend())
    shared_key = key_A.exchange(public_mix_B_ser)

    derived_key = HKDF(
        algorithm = hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    return derived_key

In [ ]:
dh_key = dh_key_exchange(parameters)
dh_key

**Action #1** -> Run all the upper cells

#### RSA (Rivest–Shamir–Adleman)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

### Data encryption

#### AES (ECB) 

In [ ]:
def encrypt_data_ecb(data, key):
      
    block_size = AES.block_size
    # TODO
    padding_length = block_size - len(data) % block_size # Calculate padding_length to make len(image array) a multiple of block_size
    data += bytes(padding_length * ".", "UTF-8") # Just an arbitrary padding byte

 
    mode   = AES.MODE_ECB
    aes    = AES.new(key, mode)
    
    encrypted_data = aes.encrypt(data) # Your code to encrypt image_array using AES

    return encrypted_data

In [ ]:
shared_key_aes = dh_key_exchange(parameters)
data_enc = encrypt_data_ecb(columns_b,shared_key_aes)

#### AES (CBC)

### Data authentication

#### HMAC

Hash Algorihtms
- **SHA1** is a generic cryptographic hash function and uses 160 bit long key to encrypt data
- **SHA-256** is a patented cryptographic hash function created by the National Security Agency in 2001 as a successor to SHA-1. It's one flavor of SHA-2 (Secure Hash Algorithm 2). uses 256 bit long key to encrypt data

**Hash-based message authentication code** is a cryptographic authentication technique that uses a hash function (SHA-1, MD5, or RIPEMD-128/60) and a secret key.
A pair using this system must agree on:

- Secret keys - They must have a way to decode messages they get. A secret key handles this task, and it's meant to stay secret and hidden.
- Algorithm - They must pick one hash function that all of their messages will move through

In [ ]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import cmac, hashes, hmac
import os

#key 1: For encryption function and gives confidentiality and authencity
key_1 = dh_key_exchange(parameters)
#key 2: For hash function to verify integrity and authencity
key_2 = dh_key_exchange(parameters)

# authencity_data = b"authenticated but unencrypted data"

key_1

In [ ]:
h = hmac.HMAC(key_2, hashes.SHA256(), backend=default_backend())

h.update(ciphertext)
signature = h.finalize()

write_file(signature,'signature.txt')
signature